In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split    
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


## Testing with GradientBoost from XGBoost

In [7]:
df = pd.read_csv('../dataset/Australia_Grand_Prix_2025_data.csv')

In [8]:
df

,driver_number,driver,year,event,team,FP1_air_temp,FP1_humidity,FP1_rainfall,FP1_pressure,FP1_wind_speed,...,Q_total_time,R_air_temp,R_humidity,R_rainfall,R_pressure,R_wind_speed,R_wind_direction,R_track_temp,R_total_time,race_position
0,1,VER,2024,Australia Grand Prix,Red Bull Racing,18.469136,61.012346,0.0,1023.564198,1.262963,...,98.397042,20.622222,44.451389,0.0,1020.601389,0.953472,154.784722,38.402083,83.287416,R
1,2,SAR,2024,Australia Grand Prix,Williams,18.469136,61.012346,0.0,1023.564198,1.262963,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,RIC,2024,Australia Grand Prix,RB,18.469136,61.012346,0.0,1023.564198,1.262963,...,93.306416,20.622222,44.451389,0.0,1020.601389,0.953472,154.784722,38.402083,84.694672,12
3,4,NOR,2024,Australia Grand Prix,McLaren,18.469136,61.012346,0.0,1023.564198,1.262963,...,92.064222,20.622222,44.451389,0.0,1020.601389,0.953472,154.784722,38.402083,83.213935,3
4,10,GAS,2024,Australia Grand Prix,Alpine,18.469136,61.012346,0.0,1023.564198,1.262963,...,91.227155,20.622222,44.451389,0.0,1020.601389,0.953472,154.784722,38.402083,85.131864,13
5,11,PER,2024,Australia Grand Prix,Red Bull Racing,18.469136,61.012346,0.0,1023.564198,1.262963,...,97.995342,20.622222,44.451389,0.0,1020.601389,0.953472,154.784722,38.402083,84.059169,5
6,14,ALO,2024,Australia Grand Prix,Aston Martin,18.469136,61.012346,0.0,1023.564198,1.262963,...,91.118187,20.622222,44.451389,0.0,1020.601389,0.953472,154.784722,38.402083,84.456398,8
7,16,LEC,2024,Australia Grand Prix,Ferrari,18.469136,61.012346,0.0,1023.564198,1.262963,...,97.876833,20.622222,44.451389,0.0,1020.601389,0.953472,154.784722,38.402083,83.144135,2
8,18,STR,2024,Australia Grand Prix,Aston Martin,18.469136,61.012346,0.0,1023.564198,1.262963,...,91.553222,20.622222,44.451389,0.0,1020.601389,0.953472,154.784722,38.402083,84.690517,6
9,20,MAG,2024,Australia Grand Prix,Haas F1 Team,18.469136,61.012346,0.0,1023.564198,1.262963,...,90.310085,20.622222,44.451389,0.0,1020.601389,0.953472,154.784722,38.402083,84.584562,10


In [9]:
# Drop columns starting with 'R' (except we keep y separately) and drop non-numeric columns
X = df.loc[:, ~df.columns.str.startswith("R")].drop(columns=['event', 'team', 'driver'], errors='ignore')
X = X.select_dtypes(include=[np.number])  # Keep only numeric columns
y = df['R_total_time']

# Remove rows where target is NaN
valid_idx = y.notna()
X = X[valid_idx]
y = y[valid_idx]

print(f"Features: {X.columns.tolist()}")
print(f"Shape: {X.shape}")

Features: ['driver_number', 'year', 'FP1_air_temp', 'FP1_humidity', 'FP1_rainfall', 'FP1_pressure', 'FP1_wind_speed', 'FP1_wind_direction', 'FP1_track_temp', 'FP1_total_time', 'FP2_air_temp', 'FP2_humidity', 'FP2_rainfall', 'FP2_pressure', 'FP2_wind_speed', 'FP2_wind_direction', 'FP2_track_temp', 'FP2_total_time', 'FP3_air_temp', 'FP3_humidity', 'FP3_rainfall', 'FP3_pressure', 'FP3_wind_speed', 'FP3_wind_direction', 'FP3_track_temp', 'FP3_total_time', 'Q_air_temp', 'Q_humidity', 'Q_rainfall', 'Q_pressure', 'Q_wind_speed', 'Q_wind_direction', 'Q_track_temp', 'Q_total_time']
Shape: (59, 34)


In [10]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = XGBRegressor(n_estimators=300, learning_rate=0.05, max_depth=3, random_state=42, verbosity=0)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.4f} seconds")
print(f"MAE: {mae:.4f} seconds")
print(f"R² Score: {r2:.4f}")

RMSE: 26.2151 seconds
MAE: 8.4719 seconds
R² Score: -0.1277
